In [1]:
!pip install selenium
!pip install openpyxl

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

chrome_options = Options()
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')

driver = webdriver.Chrome(options=chrome_options)
url = "https://newhavenct.maps.arcgis.com/apps/webappviewer/index.html?id=7d287a1263dd4fc781960069fb94c6b4"
driver.get(url) 

wait = WebDriverWait(driver, 18)
wait2 = WebDriverWait(driver, 10)

try:
    button = wait.until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="widgets_Splash_Widget_14"]/div[2]/div[2]/div[2]/button'))
    )
    button.click()
except:
    pass

time.sleep(1)

slider = wait.until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="widgets_AttributeTable_Widget_13"]/div[1]'))
)
slider.click()

time.sleep(14)

try:
    load = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dijit_form_ToggleButton_0"]')))
    driver.execute_script("arguments[0].click();", loadd)
except:
    pass

load = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dijit_form_ToggleButton_0"]')))
driver.execute_script("arguments[0].click();", load)

time.sleep(15)

scroll_counter = 0
existing_data = pd.DataFrame(columns=["Id", "OwnerName", "OwnerAddress", "OwnerCity"])
found_id = False

while not found_id:
    table_rows = wait2.until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@class="jimu-widget-attributetable-main"]//div[@data-dojo-attach-point="containerNode"]//div[@id="dgrid_0"]//div[@class="dgrid-content ui-widget-content"]//div[@role="row"]'))
    )

    for row in table_rows:
        try:
            property_id = row.find_element(By.XPATH, './/td[@class="dgrid-cell dgrid-cell-padding dgrid-column-field1 field-AV_PID field1"]').text
        except:
            property_id = np.nan
        
        try:
            owner_name = row.find_element(By.XPATH, './/td[@class="dgrid-cell dgrid-cell-padding dgrid-column-field10 field-owner_name field10"]').text
        except:
            owner_name = np.nan
        try:
            owner_address = row.find_element(By.XPATH, './/td[@class="dgrid-cell dgrid-cell-padding dgrid-column-field12 field-owner_address field12"]').text
        except:
            owner_address = np.nan
        try:
            owner_city = row.find_element(By.XPATH, './/td[@class="dgrid-cell dgrid-cell-padding dgrid-column-field14 field-owner_city field14"]').text
        except:
            owner_city = np.nan
            
        if property_id == '107547':
            found_id = True
            break
        
        if (property_id) not in existing_data['Id'].values:
            new_row = pd.DataFrame({"Id": [property_id], "OwnerName": [owner_name], "OwnerAddress": [owner_address], "OwnerCity": [owner_city]})
            existing_data = pd.concat([existing_data, new_row], ignore_index=True)
        
    scrollable_div = driver.find_element(By.XPATH, '//*[@id="dgrid_0"]/div[2]')
    driver.execute_script("arguments[0].scrollTop += 1800", scrollable_div)
    
    scroll_counter += 1 

    if scroll_counter % 3 == 0: 
        print(f"Number of scrolls: {scroll_counter}")

    time.sleep(0.6)
    
    if not table_rows or found_id:
        break

# Cerrar el navegador
driver.quit()

existing_data.drop(columns="Id", inplace=True)
existing_data.dropna(inplace=True)

Number of scrolls: 3
Number of scrolls: 6
Number of scrolls: 9
Number of scrolls: 12
Number of scrolls: 15
Number of scrolls: 18
Number of scrolls: 21
Number of scrolls: 24
Number of scrolls: 27
Number of scrolls: 30
Number of scrolls: 33
Number of scrolls: 36
Number of scrolls: 39
Number of scrolls: 42
Number of scrolls: 45
Number of scrolls: 48
Number of scrolls: 51
Number of scrolls: 54
Number of scrolls: 57
Number of scrolls: 60
Number of scrolls: 63
Number of scrolls: 66
Number of scrolls: 69
Number of scrolls: 72
Number of scrolls: 75
Number of scrolls: 78
Number of scrolls: 81
Number of scrolls: 84
Number of scrolls: 87
Number of scrolls: 90
Number of scrolls: 93
Number of scrolls: 96
Number of scrolls: 99
Number of scrolls: 102
Number of scrolls: 105
Number of scrolls: 108
Number of scrolls: 111
Number of scrolls: 114
Number of scrolls: 117
Number of scrolls: 120
Number of scrolls: 123
Number of scrolls: 126
Number of scrolls: 129
Number of scrolls: 132
Number of scrolls: 135
N

In [3]:
existing_data

,OwnerName,OwnerAddress,OwnerCity
0,CITY OF NEW HAVEN AIRPORT,165 CHURCH ST,NEW HAVEN
1,CITY OF NEW HAVEN AIRPORT,165 CHURCH ST,NEW HAVEN
2,PEREZ-RAMIREZ NOE MARTIN,199 SOUTH END RD,NEW HAVEN
3,RODRIGUEZ WILLIAM & LYSIE,11 URIAH ST,NEW HAVEN
4,GUEST CRAIG C,181 SOUTH END RD,NEW HAVEN
...,...,...,...
26938,FERRER YAMARA T,92 JUDITH TER,NEW HAVEN
26939,JOHNSON CLEMENTINE,40 RIVERVIEW RD,NEW HAVEN
26940,LUCIANO KENNETH & KEILA,30 RIVERVIEW RD,NEW HAVEN
26941,RAMOS CARLOS & ARROYO MARICEL,103 JUDITH TER,NEW HAVEN


In [4]:
existing_data.to_excel('new_haven_property.xlsx', index=False)

In [5]:
import openpyxl

workbook = openpyxl.load_workbook('new_haven_property.xlsx')
worksheet = workbook.active

for column_cells in worksheet.columns:
    max_length = 0
    column = column_cells[0].column_letter
    for cell in column_cells:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2 
    worksheet.column_dimensions[column].width = adjusted_width

workbook.save('new_haven_property.xlsx')